In [ ]:
from terra_sdk.client.lcd.api.tx import CreateTxOptions
from terra_sdk.core.wasm import MsgExecuteContract
from terra_sdk.key.mnemonic import MnemonicKey
from terra_sdk.client.lcd import LCDClient
from terra_sdk.core import Coins, Coin
import requests, json

### Initialize Client & Wallet

<sub>**Note:** _Only uncomment the code block which corresponds to the network on which you would like to carry out transactions and comment out the other code blocks. In this notebook, we utilize testnet in order to interact with a smart contract in a testing environment._</sub>

##### Testnet

In [ ]:
# Create client to communicate with testnet.
terra = LCDClient(
    url="https://bombay-lcd.terra.dev/",
    chain_id="bombay-12"
)

# Initialize wallet with associated mnemonic key.
mk = MnemonicKey(mnemonic="<INSERT MNEMONIC KEY HERE>")
wallet = terra.wallet(mk)


##### Mainnet

In [ ]:
# # Create client to communicate with mainnet.
# terra = LCDClient(
#     url="https://lcd.terra.dev",
#     chain_id="columbus-5"
# )

# # Initialize wallet with associated mnemonic key.
# mk = MnemonicKey(mnemonic="<INSERT MNEMONIC KEY HERE>")
# wallet = terra.wallet(mk)

### Set & View Gas Prices

In [ ]:
# Request current gas rates for future fee estimation.
gas_price_dict = requests.get("https://fcd.terra.dev/v1/txs/gas_prices").json()
gas_price_dict

---

## Deposit UST into Anchor

---

**Note:** _Anchor is a decentralized savings protocol which offers low-volatile yields on Terra stablecoin deposits. In this example, we will deposit 1 UST into Anchor utilizing the Terra Python SDK. You may interact with Anchor by going to the [Anchor EARN page](https://app.anchorprotocol.com/earn)._

### Set Relevant Transaction Parameters

In [ ]:
# Initialize sender wallet and smart contract addresses.
sender_address = wallet.key.acc_address
anchor_contract_address = 'terra15dwd5mj8v59wpj0wvt233mf5efdff808c5tkal'

# Set relevant parameters for transactions.
tx_options = CreateTxOptions(
    msgs=[
        MsgExecuteContract(
            sender=sender_address,
            contract=anchor_contract_address,
            execute_msg={"deposit_stable": {}},
            coins=Coins([Coin("uusd", 1000000)])
        )
    ],
    gas="auto",
    gas_prices=Coins(gas_price_dict),
    fee_denoms="uusd",
    gas_adjustment=1.5
)

### Create, Sign & Execute Transaction

In [ ]:
# Create and sign transaction.
tx = wallet.create_and_sign_tx(options=tx_options)

# Broadcast the request for execution to the Terra node.
result = terra.tx.broadcast(tx)

### Print Transaction Information

In [ ]:
# Utilize print helper function to neatly print out transaction information.
%run helper-functions.ipynb
print_message(result)